### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle

# for building DQN model
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

# Import the environment
from Env import CabDriver

import time

Using TensorFlow backend.


#### Defining Time Matrix

In [2]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

#### Tracking the state-action pairs for checking convergence


In [3]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [4]:
class DQNAgent:
    def __init__(self, state_vec_size, action_space_size):
        # Define size of state and action
        self.state_vec_size = state_vec_size
        self.action_space_size = action_space_size

        # Write here: Specify you hyper parameters for the DQN
        self.discount_factor = 1.0
        self.learning_rate = 0.01     
        self.epsilon_max = 1.0
        self.epsilon_min = 0.0
        self.epsilon_decay = -0.000015
        
        self.batch_size = 512
        
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # create main model and target model
        self.model = self.build_model()

    # approximate Q function using Neural Network
    def build_model(self):
        # Write your code here: Add layers to your neural nets       
        model = Sequential([
            Dense(64, input_dim=36, activation='relu'),
            Dense(32, activation='relu'),
            Dense(21)  
        ])
        
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary
        return model



    def get_action(self, state_vector, time):
    # Write your code here:
    # get action from model using epsilon-greedy policy
    # Decay in ε after we generate each sample from the environment    
        epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * np.exp(self.epsilon_decay*time)
    
        if np.random.rand() <= epsilon:
            # explore: choose a random action from all possible actions
            # Choose a random action from amongst 21 possible actions
            return random.randrange(self.action_space_size)
        else:
            # choose the action with the highest q(s, a)
            # the first index corresponds to the batch size, so
            # reshape state to (1, state_vec_size) so that the first index corresponds to the batch size
            state_vec = state_vector.reshape(1, self.state_vec_size)
            q_value = self.model.predict(state_vec)
            return np.argmax(q_value[0])
        
    def append_sample(self, state, action, reward, next_state, is_terminal):
    # Write your code here:
    # save sample <s,a,r,s'> to the replay memory
        self.memory.append((state, action, reward, next_state, is_terminal))
    
    
    # pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self, action_space, time):
        
        if len(self.memory) > self.batch_size and time % 12 == 0:
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            update_input = np.zeros((self.batch_size, self.state_vec_size))
            update_target = np.zeros((self.batch_size, self.state_vec_size))
            actions, rewards, is_terminals = [], [], []
            
            for i in range(self.batch_size):
                state, action, reward, next_state, is_terminal = mini_batch[i]
                
                state_encod = env.state_encod_arch1(state)
                next_state_encod = env.state_encod_arch1(next_state)
                
                update_input[i] = state_encod
                actions.append(action)
                rewards.append(reward)
                update_target[i] = next_state_encod
                is_terminals.append(is_terminal)
                
                
            # Write your code from here
            # 1. Predict the target from earlier model
            target = self.model.predict(update_input)

            # 2. Get the target for the Q-network
            target_qval = self.model.predict(update_target)

            #3. Update your 'update_input' and 'update_target' batch
            for i in range(self.batch_size):
                action = actions[i]
                action_index = action_space.index(action)
                target[i][action_index] = rewards[i] + self.discount_factor * np.max(target_qval[i])
                
                
            # 4. Fit your model and track the loss values
            self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)

    def save(self, name):
        self.model.save(name)

In [5]:
Episodes = 10

### DQN block

In [6]:
env = CabDriver()
state_vec_len = len(env.state_encod_arch1(env.state_init))
agent = DQNAgent(state_vec_len, len(env.action_space))


In [7]:
time_steps = 0
rewards_per_episode = []
episodes = []

start_time = time.time()

for episode in range(Episodes):

    # Write code here
    # Call the environment
    # Call all the initialised variables of the environment
    terminal_state = False
    score = 0

    #Call the DQN agent
    # reset at the start of each episode
    (_, _, state, _) = env.reset()

    #print(f'Start Episode:{episode}********')
    
    while not terminal_state:

        # get action for the current state and take a step in the environment
        state_vector = env.state_encod_arch1(state)
        action_index = agent.get_action(state_vector, time_steps)
        action = env.action_space[action_index]
        
        next_state, reward, is_terminal = env.step(state, action, Time_matrix)
        
        #print('-'*100)
        #print(f'Current state:{state}, action: {action}, reward:{reward}, next_state:{next_state}, is_terminal:{is_terminal}')
        # save the sample <s, a, r, s', done> to the replay memory
        agent.append_sample(state, action, reward, next_state, is_terminal)

        # train after each step
        agent.train_model(env.action_space, time_steps)

        # add reward to the total score of this episode
        score += reward
        state = next_state
        
        time_steps += 1
        
        terminal_state = is_terminal
        
    #print(f'End Episode:{episode}********')
    # store total reward obtained in this episode
    rewards_per_episode.append(score)
    episodes.append(episode)
    
elapsed_time = time.time() - start_time
print(f'Elapsed time({Episodes} espisodes): {elapsed_time/60} min')

Elapsed time(10 espisodes): 0.5744564334551493 min


# Time analysis
Elapsed time(10 episodes): 7.48 min... 1000 episodes: 748.73 mins i.e. 12 hrs... 5000 episodes: 60 hrs 

Elapsed time(10 espisodes with 100 timestep interval): 0.082 min... 1000 episodes: 8.25 mins...5000 episodes: 41.24 mins

Elapsed time(10 espisodes with 50 timestep interval): 0.15 min... 1000 episodes: 15 mins... 5000 episodes: 75 mins

Elapsed time(10 espisodes with 12 timestep interval): 0.57 min... 1000 episodes: 57 mins... 5000 episodes: 285 mins ~ 5 hrs

# States to observe
Current state:(2, 0.0, 1), action: (2, 0), reward:24.0, next_state:(0, 6.0, 1), is_terminal:False

Current state:(1, 20.0, 0), action: (4, 2), reward:-20.0, next_state:(2, 0.0, 1), is_terminal:False

Current state:(4, 16.0, 4), action: (2, 1), reward:30.0, next_state:(1, 4, 5), is_terminal:False

Current state:(2, 19.0, 2), action: (0, 4), reward:-40.0, next_state:(4, 3.0, 3), is_terminal:False

Current state:(2, 22.0, 5), action: (0, 0), reward:-5, next_state:(2, 23.0, 6), is_terminal:False


### Tracking Convergence

#### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
time = np.arange(0,10000)
epsilon = []
for i in range(0,10000):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()

In [ ]:
time = np.arange(0,500000)
epsilon = []
for i in range(0,500000):
    epsilon.append(0 + (1 - 0) * np.exp(-0.000015*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()

In [ ]:
epsilon_max = 1.0
epsilon_decay = -0.000015
epsilon_min = 0.00

In [ ]:
time = np.arange(0,500000)
epsilon = []
for i in range(0,500000):
    epsilon.append(epsilon_min + (epsilon_max - epsilon_min) * np.exp(epsilon_decay*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()